In [ ]:
# 检测 chat_rag.py 文件
file_path = "../app_image.py"
# file_path = r"C:\Users\A\Project_InputCheck\test.py"
!flake8 {file_path} --max-line-length=240
!pylint {file_path}

In [ ]:
%%writefile ..\app.py
# pylint: disable=no-member  # Project structure requires dynamic path handling
"""
For more information on `huggingface_hub` Inference API support
please check the docs: https://huggingface.co/docs/huggingface_hub/v0.22.2/en/guides/inference
"""
import os
import sys
import gradio as gr
import cv2
from pyzbar.pyzbar import decode
import numpy as np
from google import genai
from google.genai import types
from dotenv import load_dotenv

# ===== 2. 初始化配置 =====
# 获取当前文件所在目录的绝对路径
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # 在 Jupyter Notebook 环境中
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

with open(
    os.path.join(current_dir, "system_role_prompt.md"), "r", encoding="utf-8"
) as f:
    system_role = f.read()


BEGIN_PROMPT = """
总结一下最新的内容
"""


CONFIRM_PROMPT = """
对比一下两个版本的差异
"""

load_dotenv(dotenv_path=os.path.join(current_dir, ".env"))  # current_dir + "\.env")
api_key = os.getenv("GEMINI_API_KEY")
gemini_client = None
if api_key:
    gemini_client = genai.Client(api_key=api_key)

MODEL_NAME = "gemini-2.0-flash-exp"


# Common safety settings for all requests
def get_safety_settings():
    return [
        types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_CIVIC_INTEGRITY", threshold="OFF")
    ]


def format_content(role: str, text: str) -> types.Content:
    """格式化单条消息内容"""
    return types.Content(
        role=role,
        parts=[types.Part(text=text)]
    )


def respond(
    message,
    history: list[tuple[str, str]],
    use_system_message,
):
    # 构建对话历史
    def build_contents(message=None, before_message=None):
        contents = []
        for val in history:
            if val["content"] == "开始":
                context = BEGIN_PROMPT
            elif val["content"] == "确认":
                context = CONFIRM_PROMPT
            else:
                context = val["content"]
            contents.append(format_content(
                val["role"],
                context
            ))

        if before_message:
            contents.append(format_content(
                "assistant",
                before_message
            ))

        if message:
            contents.append(format_content(
                "user",
                message
            ))
        return contents
    if message == "开始" and not history:
        message = BEGIN_PROMPT

    if message == "确认" and len(history) == 2:
        message = CONFIRM_PROMPT

    if message:
        # 处理普通消息
        contents = build_contents(message)
        response = ""
        if use_system_message:
            config = types.GenerateContentConfig(
                system_instruction=system_role,
                safety_settings=get_safety_settings(),
            )
        else:
            config = types.GenerateContentConfig(
                safety_settings=get_safety_settings(),
            )
        for chunk in gemini_client.models.generate_content_stream(
            model=MODEL_NAME,
            contents=contents,
            config=config
        ):
            if chunk.text:  # Check if chunk.text is not None
                response += chunk.text
                yield response


def get_gradio_version():
    return gr.__version__


game_state = {"gr_version": get_gradio_version()}  # 示例 game_state


def process_qr_frame(frame):
    """处理视频帧，检测和解码二维码"""
    if frame is None:
        return frame, game_state

    # 转换图像格式确保兼容性
    if isinstance(frame, np.ndarray):
        img = frame.copy()
    else:
        img = np.array(frame).copy()

    # 检测二维码
    decoded_objects = decode(img)

    if decoded_objects:
        # 获取二维码数据
        qr_data = decoded_objects[0].data.decode('utf-8')

        # 解析二维码数据
        qr_info = {"设定": qr_data}

        game_state.update(qr_info)

        # 在图像上绘制识别框和状态
        points = decoded_objects[0].polygon
        if points:
            # 绘制绿色边框表示成功识别
            pts = np.array(points, np.int32)
            pts = pts.reshape((-1, 1, 2))
            cv2.polylines(img, [pts], True, (0, 255, 0), 2)

            # 添加文本显示已更新
            cv2.putText(img, "QR Code Updated!", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return img, game_state


def export_chat(history):
    export_string = ""
    for item in history:
        user_msg = item[0]['content']
        bot_msg = item[1]
        export_string += f"User: {user_msg}\nBot: {bot_msg}\n\n"
    return export_string


def get_chat_history(chatbot_component):
    return chatbot_component.load_history()


with gr.Blocks(theme="soft") as demo:
    if gemini_client:
        chatbot = gr.ChatInterface(
            respond,
            title="知识助理",
            type="messages",
            additional_inputs=[
                gr.Checkbox(value=False, label="Use system message"),
            ],

        )
    else:
        gr.Markdown("Gemini API key not found. Please check your .env file.")

    with gr.Accordion("查看状态", open=False):
        game_info_image = gr.Image(label="二维码设定",
                                   webcam_constraints={
                                        "video": {
                                            "facingMode": {"ideal": "environment"}
                                        }
                                    })
        output_img = gr.Image(label="识别结果")
        game_state_output = gr.JSON(value=game_state)  # 初始显示 game_state

        # 设置实时流处理
        game_info_image.upload(
            fn=process_qr_frame,
            inputs=[game_info_image],
            outputs=[output_img, game_state_output],
            show_progress=False,
        )
        # 设置实时流处理
        game_info_image.stream(
            fn=process_qr_frame,
            inputs=[game_info_image],
            outputs=[output_img, game_state_output],
            show_progress=False,
            stream_every=0.5  # 每0.5秒处理一次
        )

if __name__ == "__main__":
    cert_file = os.path.join(current_dir, "localhost+1.pem")
    key_file = os.path.join(current_dir, "localhost+1-key.pem")

    if os.path.exists(cert_file) and os.path.exists(key_file):
        demo.launch(
            server_name="0.0.0.0",
            ssl_certfile=cert_file,
            ssl_keyfile=key_file
        )
    else:
        demo.launch(server_name="0.0.0.0")


In [ ]:
%%writefile ..\app_image.py
# pylint: disable=no-member  # Project structure requires dynamic path handling
"""
whisk逆向图片生成
"""
import os
import sys
import hashlib
import json
from datetime import datetime
from typing import Optional, List, Dict
from requests.exceptions import HTTPError
from dotenv import load_dotenv
import gradio as gr

# ===== 2. 初始化配置 =====
# 获取当前文件所在目录的绝对路径
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # 在 Jupyter Notebook 环境中
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

from Module.Common.scripts.llm.utils.google_whisk import (
    generate_image_base64,
    generate_caption,
    generate_image_fx,
    generate_story_board,
    DEFAULT_STYLE_PROMPT_DICT,
    DEFAULT_HEADERS,
    AspectRatio,
    Category
)
from Module.Common.scripts.common.auth_manager import (
    AuthKeeper,
    sustain_auth
)
from Module.Common.scripts.datasource_feishu import FeishuClient

CONFIG_PATH = os.path.join(current_dir, "config.json")

# 加载配置文件
with open(CONFIG_PATH, 'r', encoding='utf-8') as config_file:
    CONFIG = json.load(config_file)

# 修改后的常量定义
IMAGE_NUMBER = CONFIG["image_generation"]["default_image_number"]
ASPECT_RATIO = AspectRatio[CONFIG["image_generation"]["aspect_ratio"]]
SERVER_PORT = CONFIG["server"]["port"]
ERROR_PATTERNS = CONFIG["error_patterns"]
FEISHU_RECEIVE_ID = CONFIG["feishu"]["receive_id"]

# 缓存文件路径
CACHE_DIR = os.path.join(current_dir, CONFIG["cache"]["dir"])
CAPTION_CACHE_FILE = os.path.join(CACHE_DIR, CONFIG["cache"]["caption_file"])
STORY_CACHE_FILE = os.path.join(CACHE_DIR, CONFIG["cache"]["story_file"])
IMAGE_CACHE_DIR = os.path.join(CACHE_DIR, CONFIG["cache"]["image_dir"])


load_dotenv(os.path.join(current_dir, ".env"))
app_id = os.getenv("FEISHU_APP_ID", "")
app_secret = os.getenv("FEISHU_APP_SECRET", "")

# 改进后的飞书客户端初始化（更Pythonic的写法）
FEISHU_CLIENT = None
FEISHU_ENABLED = False
if app_id and app_secret:  # 先做基础检查
    try:
        FEISHU_CLIENT = FeishuClient(app_id, app_secret)
        FEISHU_CLIENT.get_access_token()  # 主动验证凭证有效性
        FEISHU_ENABLED = True
    except (ValueError, RuntimeError) as e:
        print(f"⚠️ 飞书通知功能已禁用: {str(e)}")
else:
    print("⚠️ 飞书通知功能已禁用: 缺少FEISHU_APP_ID或FEISHU_APP_SECRET环境变量")


class WhiskCache:
    """Whisk缓存管理类"""
    def __init__(self):
        os.makedirs(IMAGE_CACHE_DIR, exist_ok=True)
        self.caption_cache = self._load_cache(CAPTION_CACHE_FILE)
        self.story_cache = self._load_cache(STORY_CACHE_FILE)

    def _load_cache(self, file_path: str) -> Dict:
        """加载缓存文件"""
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}

    def save_cache(self, file_path: str, data: Dict):
        """保存缓存到文件"""
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

    def get_caption(self, image_base64: str):
        """获取缓存的图片描述"""
        hash_key = hashlib.md5(image_base64.encode()).hexdigest()
        return self.caption_cache.get(hash_key)

    def save_caption(self, image_base64: str, caption: str):
        """缓存图片描述"""
        hash_key = hashlib.md5(image_base64.encode()).hexdigest()
        self.caption_cache[hash_key] = caption
        self.save_cache(CAPTION_CACHE_FILE, self.caption_cache)

    def get_story_prompt(self, caption: str, style_key: str, additional_text: str):
        """获取缓存的故事提示词"""
        cache_key = hashlib.md5(f"{caption}_{style_key}_{additional_text}".encode()).hexdigest()
        return self.story_cache.get(cache_key)

    def save_story_prompt(self, caption: str, style_key: str, additional_text: str, prompt: str):
        """缓存故事提示词"""
        cache_key = hashlib.md5(f"{caption}_{style_key}_{additional_text}".encode()).hexdigest()
        self.story_cache[cache_key] = prompt
        self.save_cache(STORY_CACHE_FILE, self.story_cache)


class WhiskService:
    """Whisk服务类"""
    def __init__(self):
        self.cache = WhiskCache()
        self.auth_keeper = AuthKeeper(
            config_path=os.path.join(current_dir, "auth_config.json"),
            default_headers=DEFAULT_HEADERS
        )
        # 保持原始装饰器调用方式
        self.sustain_cookies = sustain_auth(self.auth_keeper, 'cookies')
        self.sustain_token = sustain_auth(self.auth_keeper, 'auth_token')

    @property
    def generate_caption_wrapped(self):
        """保持装饰器调用方式不变"""
        return self.sustain_cookies(self._generate_caption_impl)

    def _generate_caption_impl(
        self,
        image_base64: str,
        category: Category = Category.CHARACTER,  # 补全参数
        cookies: str = None  # 保持装饰器参数
    ) -> Dict:
        return generate_caption(
            image_base64=image_base64,
            category=category,  # 传递补全参数
            cookies=cookies
        )

    @property
    def generate_story_board_wrapped(self):
        """保持装饰器调用方式不变"""
        return self.sustain_cookies(self._generate_story_board_impl)

    def _generate_story_board_impl(
        self,
        characters: Optional[List[str]] = None,
        style_prompt: Optional[str] = None,
        location_prompt: Optional[str] = None,  # 补全参数
        pose_prompt: Optional[str] = None,       # 补全参数
        additional_input: str = "",
        cookies: str = None
    ) -> Dict:
        return generate_story_board(
            characters=characters,
            style_prompt=style_prompt,
            location_prompt=location_prompt,  # 传递补全参数
            pose_prompt=pose_prompt,           # 传递补全参数
            additional_input=additional_input,
            cookies=cookies
        )

    @property
    def generate_image_fx_wrapped(self):
        """保持装饰器调用方式不变"""
        return self.sustain_token(self._generate_image_fx_impl)

    def _generate_image_fx_impl(
        self,
        prompt: str,
        seed: Optional[int] = None,
        aspect_ratio: AspectRatio = AspectRatio.LANDSCAPE,
        output_prefix: str = None,
        image_number: int = 4,
        auth_token: str = None,
        save_local: bool = False  # 补全参数
    ) -> List:
        return generate_image_fx(
            prompt=prompt,
            seed=seed,
            aspect_ratio=aspect_ratio,
            output_prefix=output_prefix,
            image_number=image_number,
            auth_token=auth_token,
            save_local=save_local  # 传递补全参数
        )

    def generate_images(
        self,
        image_input1: str,
        image_input2: str,
        style_key: str,
        additional_text: str
    ):
        """处理图片生成请求"""
        # 基础参数准备
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        current_date = datetime.now().strftime("%Y%m%d")
        output_prefix = os.path.join(
            IMAGE_CACHE_DIR,
            f"generated_image_{current_date}_take{len(os.listdir(IMAGE_CACHE_DIR)) + 1}"
        )

        # 处理指令直出模式
        if additional_text.startswith('/img') and len(additional_text) > 4:
            clean_prompt = additional_text[4:].strip()
            if clean_prompt:
                return self._handle_direct_mode(clean_prompt, current_time, output_prefix)

        # 输入检查
        if not any([image_input1, image_input2]):
            return None, None

        # 预处理图片数据
        image_data = self._prepare_image_data(image_input1, image_input2)
        all_caption_cached = all(caption for _, caption in image_data)

        # 打印缓存状态
        self._print_cache_status(current_time, all_caption_cached, image_data, style_key, additional_text)

        try:
            # 生成描述和故事板
            captions = self._process_captions(image_data)
            if not captions:
                return None, None

            # 生成最终提示词
            final_prompt = self._get_final_prompt(captions, style_key, additional_text)
            print(f"最终Prompt: \n{final_prompt}")

            return self._generate_final_images(final_prompt, output_prefix)

        except HTTPError as e:
            return self._handle_http_error(e)
        except Exception as e:
            print(f"其他图片生成错误: {str(e)}")
            return None, None

    def _handle_direct_mode(self, clean_prompt, current_time, output_prefix):
        """处理指令直出模式"""
        print(f"[{current_time}] 指令直出，使用提示词: \n{clean_prompt}")
        try:
            image_files = self.generate_image_fx_wrapped(
                prompt=clean_prompt,
                output_prefix=output_prefix,
                image_number=2
            )
            return self._format_image_output(image_files)
        except HTTPError as e:
            return self._handle_http_error(e)
        except Exception as e:
            print(f"其他图片生成错误: {str(e)}")
            return None, None

    def _prepare_image_data(self, *image_inputs):
        """预处理图片数据"""
        image_data = []
        for img in image_inputs:
            if img:
                base64_str = generate_image_base64(img)
                cached_caption = self.cache.get_caption(base64_str)
                image_data.append((base64_str, cached_caption))
        return image_data

    def _print_cache_status(self, current_time, all_caption_cached, image_data, style_key, additional_text):
        """打印缓存状态"""
        story_prompt_cached = False
        if all_caption_cached:
            caption_text = "|".join(caption for _, caption in image_data)
            story_prompt_cached = self.cache.get_story_prompt(caption_text, style_key, additional_text) is not None

        print(
            f"\n[{current_time}] 生成图片 | "
            f"素材提示词缓存: {'启用' if all_caption_cached else '未启用'} | "
            f"最终提示词缓存: {'启用' if story_prompt_cached else '未启用'}"
        )

    def _process_captions(self, image_data):
        """处理图片描述"""
        captions = []
        for base64_str, cached_caption in image_data:
            if not cached_caption:
                new_caption = self.generate_caption_wrapped(base64_str)
                if new_caption:
                    self.cache.save_caption(base64_str, new_caption)
                    captions.append(new_caption)
            else:
                captions.append(cached_caption)
        return captions

    def _get_final_prompt(self, captions, style_key, additional_text):
        """获取最终提示词"""
        final_prompt = self.cache.get_story_prompt(
            "|".join(captions),
            style_key,
            additional_text
        )
        if not final_prompt:
            final_prompt = self.generate_story_board_wrapped(
                characters=captions,
                style_prompt=DEFAULT_STYLE_PROMPT_DICT.get(style_key, ""),
                additional_input=additional_text
            )
            if final_prompt:
                self.cache.save_story_prompt(
                    "|".join(captions),
                    style_key,
                    additional_text,
                    final_prompt
                )
        return final_prompt

    def _generate_final_images(self, final_prompt, output_prefix):
        """生成最终图片"""
        image_files = self.generate_image_fx_wrapped(
            prompt=final_prompt,
            output_prefix=output_prefix,
            image_number=IMAGE_NUMBER
        )
        return self._format_image_output(image_files)

    def _format_image_output(self, image_files):
        """格式化图片输出"""
        return (
            image_files[0] if image_files else None,
            image_files[1] if len(image_files) > 1 else None
        )

    def _handle_http_error(self, e):
        """统一处理HTTP错误"""
        error_info = {
            "status_code": e.response.status_code,
            "reason": e.response.reason,
            "url": e.response.url,
            "response_text": e.response.text[:200]
        }
        error_source = self._detect_error_source(e.response.url)

        if error_info['status_code'] == 401:
            print(f"[{error_source}] 自动续签认证失败，详细见飞书")
            if FEISHU_ENABLED:
                FEISHU_CLIENT.send_message(
                    receive_id=FEISHU_RECEIVE_ID,
                    content={"text": "Whisk的Cookie已过期，请及时续签"},
                    msg_type="text"
                )
        elif error_info['status_code'] == 400:
            print(f"[{error_source}] 业务和谐，计划重试...")
            gr.Warning("图片生成失败，请换一张再试试")
        else:
            print(f"[{error_source}] 其他HTTP错误，添加一个飞书消息...")

        return None, None

    def _detect_error_source(self, url):
        """优化后的错误来源检测（直观简洁版）"""
        return next(
            (v for k, v in ERROR_PATTERNS.items() if k in url),
            '未知来源'
        )


# 创建服务实例
whisk_service = WhiskService()

# Gradio界面
with gr.Blocks(theme="soft") as demo:
    with gr.Row():
        with gr.Column(scale=1):
            with gr.Row():
                input_image1 = gr.Image(
                    label="上传图片1",
                    type="filepath",
                    height=300
                )
                input_image2 = gr.Image(
                    label="上传图片2（可选）",
                    type="filepath",
                    height=300
                )
            style_dropdown = gr.Dropdown(
                choices=list(DEFAULT_STYLE_PROMPT_DICT.keys()),
                value=list(DEFAULT_STYLE_PROMPT_DICT.keys())[0],
                label="选择风格"
            )
            additional_text_ui = gr.Textbox(
                label="补充提示词",
                placeholder="请输入额外的提示词...",
                lines=3
            )
            generate_btn = gr.Button("生成图片")

        with gr.Column(scale=2):
            output_image1 = gr.Image(label="生成结果 1")
            output_image2 = gr.Image(label="生成结果 2")

    generate_btn.click(
        fn=whisk_service.generate_images,
        inputs=[input_image1, input_image2, style_dropdown, additional_text_ui],
        outputs=[output_image1, output_image2]
    )

if __name__ == "__main__":
    demo.launch(
        server_name=CONFIG["server"]["name"],
        server_port=SERVER_PORT,      # 使用配置值
        ssl_verify=CONFIG["server"]["ssl_verify"],
        share=CONFIG["server"]["share"],
        allowed_paths=[IMAGE_CACHE_DIR]
    )


In [ ]:
valid_json_str = '{"prompt": "A whimsical close-up shot of a bento box. Inside, a miniature young woman, crafted from colored rice and vegetables, with a single sesame seed for an eye, is shown saluting.  Her kimono, rendered in shades of pastel red with a white floral pattern, has long, flowing sleeves.  A tiny pink heart-shaped sign with miniature Japanese characters is in front of her.  The woman\'s dark hair is neatly pulled back, adorned with a small pink flower. Her expression is pleasant. The bento box sits on a table, the surrounding environment outside the box is not visible. The overall style is kawaii, with soft pastel colors and delicate details. The lighting is soft and diffused.  All elements are miniature and made of edible foods, kept entirely within the bento box"}'
import json
# 可以成功解析
parsed = json.loads(valid_json_str)
json.dumps(parsed)
# print(parsed["prompt"][:30])  # 输出前30个字符验证

In [ ]:
# 在这里，您可以通过 ‘args’  获取节点中的输入变量，并通过 'ret' 输出结果
# 'args' 和 'ret' 已经被正确地注入到环境中
# 下面是一个示例，首先获取节点的全部输入参数params，其次获取其中参数名为‘input’的值：
# params = args.params;
# input = params.input;
# 下面是一个示例，输出一个包含多种数据类型的 'ret' 对象：
# ret: Output =  { "name": ‘小明’, "hobbies": [“看书”, “旅游”] };
import requests_async as requests

# 公共配置
DEFAULT_HEADERS = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br, zstd',
    'accept-language': 'zh-CN,zh;q=0.9,ja;q=0.8,zh-TW;q=0.7,en;q=0.6',
    'cache-control': 'no-cache',
    'content-type': 'text/plain;charset=UTF-8',
    'origin': 'https://labs.google',
    'pragma': 'no-cache',
    'priority': 'u=1, i',
    'referer': 'https://labs.google/',
    'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
    'x-client-data': 'CKu1yQEIlLbJAQiitskBCKmdygEIuoXLAQiVocsBCIqjywEIhaDNAQjU284BCJDfzgEYj87NAQ=='
}

DEFAULT_STYLE_PROMPT_DICT = {
    "贺卡": """
A rectangular shaped Valentine's Day card depicting a layered papercut diorama of a dreamy, romantic scene at sunset with a cute rim embellished with ribbons.
Scene and subject should be made out of paper, ribbons, and stickers. No other materials allowed. Incorporate ALL SUBJECTS and ALL DETAILS FROM THE SCENE.

The color palette is soft and romantic, with pastel pinks, purples, and oranges dominating the sky.
The figures are small and cute, and the overall style is whimsical and charming.
The card is heavily embellished with glitter, adding to the romantic and festive atmosphere.

The final output should look like a card that was handcrafted with love and include a cute Valentine's day message.
""",
    "巧克力": """
A Valentine's day box of chocolates is shown in a close-up shot.
The chocolates are arranged in a dark brown plastic tray with individual compartments.
The chocolates are various shapes and sizes, and are decorated with different colors and patterns.
The lighting is soft and diffused, and the overall tone of the image is warm and inviting.
The image has a slightly desaturated look, with muted colors and a slightly grainy texture.

Exquisitely sculpt all characters as a cute, small, round, chocolate nestled into the box.
If provided a scene or environment, the chocolates are painted and made into creative shapes to depict elements of the scene in detail SEPARATE from the characters.
The characters are positioned within the chocolate box, surrounded by other beautiful chocolates, creating a visually appealing and balanced composition.
The final product should show ONLY a close up of the exquisite and colorful box of decorated chocolates.
""",
    "便当": """
A close-up shot of a super cute bento box scene featuring miniature, real, colorful Japanese foods and veggies.
Characters have single sesame seeds for eyes and are crafted out of different colored rice and veggies to be tiny, kawaii pieces nestled INTO the box.
The characters are positioned centrally within the bento box, surrounded by other miniature food items, creating a visually appealing and balanced composition.

Elements must be mini and created by edible foods only and always kept INSIDE the bento box.
If provided, incorporate a scene/environment OUTSIDE of the box. Make it such that the box sits on a table in that location.
The overall style is whimsical and charming, with a focus on soft, pastel colors and delicate details. The lighting is soft and diffused, enhancing the delicate textures and colors.

The final product should show ONLY a close up of the exquisite bento box sitting on a surface of a given location.
""",
}


def generate_imagen3_character_prompt(
    characters_descriptions=None,
    location=None,
    style_description=None,
    pose_description=None,
    user_instructions=None
) -> str:
    prompt_parts = []

    # 添加基础开头
    base_start = """你是一个提示词大师，请根据下面的信息制作一个提供给图片生成模型的提示词。
Instructions:
1. You will be creating a prompt for a text-to-image model that will be placing characters/subjects into a location background.
2. You will make sure to concisely describe each character/subject and what they are doing and will not lose track of the characters'/subjects' visual details.
3. You will make sure the background is the one provided by the location prompt."""
    prompt_parts.append(base_start)

    # 处理角色描述
    if characters_descriptions and len(characters_descriptions) > 0:
        character_template = """
There are {num_characters} characters/subjects in the scene. Make sure to ignore the locations from these character/subject prompts and only extract the character/subject:
{character_prompts}"""

        character_prompts = []
        for i, desc in enumerate(characters_descriptions, 1):
            character_prompts.append(f"Character/Subject {i} description: '{desc}'")
        formatted_characters = "\n".join(character_prompts)

        prompt_parts.append(character_template.format(
            num_characters=len(characters_descriptions),
            character_prompts=formatted_characters
        ))

    # 处理位置
    if location:
        location_template = """
This is the location used for the background (Ignore the locations in characters'/subjects' prompt above!)
It is critical to use this exact location: '{location}'"""
        prompt_parts.append(location_template.format(location=location))

    # 处理风格
    if style_description:
        style_template = """
This is the aesthetic/visual/artistic style to use (ignore any other style or visual aesthetic mentioned above);
ONLY extract the aesthetic/visual/artistic styles from this prompt and nothing about the subjects/location/objects/etc. within it:
'{style}'

Make sure to rewrite the prompt to conform the character/subject/location/etc. to these colors and visual styles and produce a strong style transfer.
Emphasize the updated styles early in the prompt and if applicable, include them as parts of the character/subject descriptions.
It's critical to use this exact style unless the user says otherwise.
When describing the scene and colors, it's critical that they conform to this style."""
        prompt_parts.append(style_template.format(style=style_description))

    # 处理姿势
    if pose_description:
        pose_template = """
This is the pose to use (ignore any other pose mentioned above);
ONLY extract the pose from this prompt and nothing about the subjects/location/style/etc. within it:
'{pose}'

Make sure to rewrite the prompt to conform to these poses and produce a pose transfer in a way that makes sense for the user's chosen subject.
Describe the pose of the character/subject in the image in detail so someone could draw it without the original reference.
Do not detail any of the actual objects or subjects in the scene. Purely describe the pose and action.
Emphasize the updated poses early in the prompt, but do so only concisely.
Make sure to overwrite any other poses mentioned above unless the user says otherwise."""
        prompt_parts.append(pose_template.format(pose=pose_description))

    # 处理用户指令
    if user_instructions:
        instruction_template = """
User Instructions: '{instructions}'"""
        prompt_parts.append(instruction_template.format(instructions=user_instructions))

    # 添加通用的结尾部分
    ending_template = """
Describe this picture in great detail, thinking of all the details someone would need to recreate that picture without seeing it.
Make sure to not exceed more than 200 words in this prompt.
Do not include details that do not conform to the user's specified characters/subjects, location, style, or pose.

Don't generate images, just write text. Do not prompt the user for more information. Only return the new prompt."""
    prompt_parts.append(ending_template)

    # 组合所有部分
    final_prompt = "\n".join(prompt_parts)
    return final_prompt


async def _send_request(
    url: str,
    payload,
    auth_token=None,
    cookies=None
):
    """统一处理HTTP请求（异步版本）"""
    headers = DEFAULT_HEADERS.copy()
    if auth_token:
        headers['authorization'] = auth_token
    if cookies:
        headers['Cookie'] = cookies
    try:
        response = await requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        return {"error": f"HTTP error: {str(e)}"}
    except Exception as e:
        return {"error": f"Request failed: {str(e)}"}


async def generate_image_fx(
    prompt: str,
    seed=None,
    aspect_ratio="IMAGE_ASPECT_RATIO_LANDSCAPE",
    image_number: int = 2,
    auth_token=None,
):
    """生成图像API封装（返回base64数据）"""
    payload = {
        'userInput': {
            'candidatesCount': image_number,
            'prompts': [prompt],
            **({'seed': seed} if seed is not None else {})
        },
        'clientContext': {'tool': 'BACKBONE'},
        'modelInput': {'modelNameType': 'IMAGEN_3_1'},
        'aspectRatio': aspect_ratio
    }

    response = await _send_request(
        'https://aisandbox-pa.googleapis.com/v1:runImageFx',
        payload,
        auth_token=auth_token
    )

    if 'error' in response:
        return response

    output = {
        "images": [],
        "metadata": {
            "prompt": prompt,
            "seed": seed,
            "aspect_ratio": aspect_ratio
        }
    }

    if 'imagePanels' in response:
        for panel in response['imagePanels']:
            for img_data in panel['generatedImages']:
                try:
                    output['images'].append({
                        "base64": img_data['encodedImage'],
                        "seed": img_data.get('seed'),
                        "format": "png"
                    })
                except KeyError as e:
                    print(f"Invalid image data format: {e}")
    return output


async def main(args: Args) -> Output:
    "提示词生成"
    # 解析输入参数
    style_key = args.params['style']
    character_list = []
    image1 = args.params['image1']
    image2 = args.params['image2']
    character_list = [image1, image2]
    style_info = DEFAULT_STYLE_PROMPT_DICT.get(style_key, DEFAULT_STYLE_PROMPT_DICT['贺卡'])
    instruction = args.params['extra']
    prompt = generate_imagen3_character_prompt(
        characters_descriptions=character_list,
        style_description=style_info,
        user_instructions=instruction
    )
    params = {"prompt": prompt}

    # 构建标准化输出
    return params


In [ ]:
from gradio_client import Client, handle_file

client = Client("http://0.0.0.0/")
prompt = "A layered papercut diorama Valentine's Day card depicting a young woman with long, dark brown hair.  She holds a bouquet of white roses and eucalyptus in a pale pink paper wrapper; a receipt is partially tucked inside.  The woman wears a sheer, long-sleeved, lavender mesh top over a solid lavender underlayer. Her light skin tone is visible. The woman and bouquet are crafted from paper, ribbons, and stickers, with pastel pinks, purples, and oranges in the background.  The style is whimsical and charming, heavily embellished with glitter. The overall aesthetic is soft, romantic, and handcrafted, resembling a cute Valentine's Day card."
additional_text = "/img " + prompt

result = client.predict(
		image_input1=handle_file(r'E:\Download\[pixiv] Trans Tribune (Wataya) - 64.jpg'),
		image_input2=None,
		style_key="贺卡",
		additional_text=additional_text,
		api_name="/generate_images"
)
print(result)

In [ ]:
import lark_oapi as lark
from lark_oapi.api.im.v1 import *
import json


# 注册接收消息事件，处理接收到的消息。
# Register event handler to handle received messages.
# https://open.feishu.cn/document/uAjLw4CM/ukTMukTMukTM/reference/im-v1/message/events/receive
def do_p2_im_message_receive_v1(data: P2ImMessageReceiveV1) -> None:
    res_content = ""
    if data.event.message.message_type == "text":
        res_content = json.loads(data.event.message.content)["text"]
    else:
        res_content = "解析消息失败，请发送文本消息\nparse message failed, please send text message"

    content = json.dumps(
        {
            "text": "收到你发送的消息："
            + res_content
            + "\nReceived message:"
            + res_content
        }
    )

    if data.event.message.chat_type == "p2p":
        request = (
            CreateMessageRequest.builder()
            .receive_id_type("chat_id")
            .request_body(
                CreateMessageRequestBody.builder()
                .receive_id(data.event.message.chat_id)
                .msg_type("text")
                .content(content)
                .build()
            )
            .build()
        )
        # 使用OpenAPI发送消息
        # Use send OpenAPI to send messages
        # https://open.feishu.cn/document/uAjLw4CM/ukTMukTMukTM/reference/im-v1/message/create
        response = client.im.v1.chat.create(request)

        if not response.success():
            raise Exception(
                f"client.im.v1.chat.create failed, code: {response.code}, msg: {response.msg}, log_id: {response.get_log_id()}"
            )
    else:
        request: ReplyMessageRequest = (
            ReplyMessageRequest.builder()
            .message_id(data.event.message.message_id)
            .request_body(
                ReplyMessageRequestBody.builder()
                .content(content)
                .msg_type("text")
                .build()
            )
            .build()
        )
        # 使用OpenAPI回复消息
        # Reply to messages using send OpenAPI
        # https://open.feishu.cn/document/uAjLw4CM/ukTMukTMukTM/reference/im-v1/message/reply
        response: ReplyMessageResponse = client.im.v1.message.reply(request)
        if not response.success():
            raise Exception(
                f"client.im.v1.message.reply failed, code: {response.code}, msg: {response.msg}, log_id: {response.get_log_id()}"
            )


# 注册事件回调
# Register event handler.
event_handler = (
    lark.EventDispatcherHandler.builder("", "")
    .register_p2_im_message_receive_v1(do_p2_im_message_receive_v1)
    .build()
)


# 创建 LarkClient 对象，用于请求OpenAPI, 并创建 LarkWSClient 对象，用于使用长连接接收事件。
# Create LarkClient object for requesting OpenAPI, and create LarkWSClient object for receiving events using long connection.
client = lark.Client.builder().app_id(lark.APP_ID).app_secret(lark.APP_SECRET).build()
wsClient = lark.ws.Client(
    lark.APP_ID,
    lark.APP_SECRET,
    event_handler=event_handler,
    log_level=lark.LogLevel.DEBUG,
)


def main():
    #  启动长连接，并注册事件处理器。
    #  Start long connection and register event handler.
    wsClient.start()


if __name__ == "__main__":
    main()


In [ ]:
import asyncio
import lark_oapi as lark
from lark_oapi.api.im.v1 import *
import json

# 原有B站代码保持不动...
# ... [你原有的B站相关代码] ...

# 飞书机器人部分改造为异步
class FeishuBot:
    def __init__(self):
        self.client = lark.Client.builder().app_id(lark.APP_ID).app_secret(lark.APP_SECRET).build()
        self.ws_client = lark.ws.Client(
            lark.APP_ID,
            lark.APP_SECRET,
            event_handler=self.event_handler(),
            log_level=lark.LogLevel.DEBUG,
        )

    def event_handler(self):
        handler = (
            lark.EventDispatcherHandler.builder("", "")
            .register_p2_im_message_receive_v1(self.do_p2_im_message_receive_v1)
            .build()
        )
        return handler

    async def handle_message(self, data: P2ImMessageReceiveV1):
        # 消息处理逻辑（保持原有逻辑）
        res_content = json.loads(data.event.message.content)["text"] if data.event.message.message_type == "text" else "请发送文本"

        content = json.dumps({"text": f"收到：{res_content}\nB站最新动态数：{len(result)}"})  # 结合B站数据

        if data.event.message.chat_type == "p2p":
            req = CreateMessageRequest.builder().receive_id_type("chat_id").request_body(
                CreateMessageRequestBody.builder()
                .receive_id(data.event.message.chat_id)
                .msg_type("text")
                .content(content)
                .build()
            ).build()
            response = self.client.im.v1.chat.create(req)
        else:
            req = ReplyMessageRequest.builder().message_id(data.event.message.message_id).request_body(
                ReplyMessageRequestBody.builder().content(content).msg_type("text").build()
            ).build()
            response = self.client.im.v1.message.reply(req)

        if not response.success():
            print(f"消息发送失败: {response.code} - {response.msg}")

    def do_p2_im_message_receive_v1(self, data: P2ImMessageReceiveV1):
        # 将同步回调包装为异步任务
        asyncio.create_task(self.handle_message(data))

# 异步运行飞书机器人
async def run_feishu_bot():
    bot = FeishuBot()
    # 由于SDK同步方法，使用to_thread避免阻塞
    await asyncio.to_thread(bot.ws_client.start)

# 组合运行B站任务和飞书机器人
async def main():
    飞书任务 = run_feishu_bot()
    await asyncio.gather(飞书任务)

# 在Jupyter中直接运行（需要顶层await支持）
try:
    # 如果已经存在运行中的循环
    loop = asyncio.get_event_loop()
    if loop.is_running():
        create_task = loop.create_task(main())
    else:
        asyncio.run(main())
except KeyboardInterrupt:
    pass
finally:
    print("服务已停止")

In [ ]:
"""
飞书机器人服务模块

该模块实现了一个基于飞书开放平台的机器人服务，主要功能包括：
- 消息接收与回复
- 图片、音频等多媒体处理
- AI图像生成与处理
- 定时任务调度
- 配置文件管理

依赖:
- lark_oapi: 飞书开放平台SDK
- gradio_client: Gradio客户端，用于AI服务调用
- schedule: 定时任务调度
"""

import schedule
import time
import lark_oapi as lark
from lark_oapi.api.im.v1 import *
import os
import sys
import json
import asyncio
import random
import subprocess
import shutil
from pathlib import Path
from dotenv import load_dotenv
from gradio_client import Client
import datetime
import base64
import requests
import tempfile

# 初始化配置
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

load_dotenv(os.path.join(current_dir, ".env"))
gradio_client = Client(f"http://{os.getenv('SERVER_ID', '')}/")
lark.APP_ID = os.getenv("FEISHU_APP_MESSAGE_ID", "")
lark.APP_SECRET = os.getenv("FEISHU_APP_MESSAGE_SECRET", "")

UPDATE_CONFIG_TRIGGER = "whisk令牌"
SUPPORTED_VARIABLES = ["cookies", "auth_token"]
CONFIG_FILE_PATH = os.path.join(current_dir, "auth_config.json")
custom_ffmpeg = r"C:\Users\A\AppData\Local\JianyingPro\Apps\6.3.0.12011\ffmpeg.exe"
CACHE_DIR = os.path.join(current_dir, "cache")
PROCESSED_EVENTS_FILE = os.path.join(CACHE_DIR, "processed_events.json")

COZE_ACCESS_TOKEN = os.getenv("COZE_API_KEY")

# 配置参数（替换为你的实际参数）
COZE_API_BASE = "https://api.coze.cn"
BOT_ID = "7316745484305989638"  # 替换为你的 Bot ID
VOICE_ID = "peach"  # 中文女声音色

class CozeTTS:
    def __init__(self, api_base: str, workflow_id: str, access_token: str):
        self.api_base = api_base
        self.workflow_id = workflow_id
        self.access_token = access_token

    def generate(self, text: str) -> Optional[bytes]:
        """返回音频字节流，不保存文件"""
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Content-Type": "application/json"
        }
        payload = {
            "workflow_id": self.workflow_id,
            "parameters": {
                "input": text,
                "voicebranch": VOICE_ID,
                "voice_type": "zh_female_gufengshaoyu_mars_bigtts"
            }
        }

        try:
            # 第一步：获取音频URL

            # print(f"payload: {payload}")
            # print(f"headers: {headers}")
            # print(f"url: {self.api_base}")
            response = requests.post(self.api_base, headers=headers, json=payload)
            response.raise_for_status()

            if response.status_code == 200:
                result = response.json()
                if result.get("code") == 0:
                    data = json.loads(result.get("data", "{}"))
                    # 优先使用gaoleng源
                    # Define URL mappings for different audio sources
                    url_mappings = {
                        "gaoleng": "url",
                        "speech": "link",
                        "ruomei": "url"
                    }
                    # Find the first valid audio URL
                    audio_url = None
                    for source, url_key in url_mappings.items():
                        if data.get(source):
                            audio_url = data[source].get(url_key)
                            if audio_url:
                                break

                    if audio_url:
                        # 第二步：下载音频
                        audio_response = requests.get(audio_url)
                        audio_response.raise_for_status()
                        return audio_response.content
            return None
        except Exception as e:
            print(f"TTS流程失败: {e}")
            return None

# 初始化（放在全局作用域）
coze_tts = CozeTTS(
    api_base="https://api.coze.cn/v1/workflow/run",
    workflow_id="7473130113548402722",
    access_token=COZE_ACCESS_TOKEN
)


def load_processed_events() -> dict:
    """
    加载并清理过期事件缓存

    Returns:
        dict: 事件ID到时间戳的映射字典
    """
    try:
        if os.path.exists(PROCESSED_EVENTS_FILE):
            with open(PROCESSED_EVENTS_FILE, "r", encoding="utf-8") as f:
                raw_data = json.load(f)

            if isinstance(raw_data, list):
                return {k: time.time() for k in raw_data}
            else:
                cutoff = time.time() - 32 * 3600
                return {k: float(v) for k, v in raw_data.items() if float(v) > cutoff}

    except Exception as e:
        print(f"缓存加载失败: {str(e)}")
    return {}

def save_processed_events():
    """原子化保存已处理事件ID到缓存文件"""
    try:
        os.makedirs(CACHE_DIR, exist_ok=True)
        temp_file = PROCESSED_EVENTS_FILE + ".tmp"
        with open(temp_file, "w", encoding="utf-8") as f:
            processed_events = {k: str(v) for k, v in _processed_event_ids.items()}
            json.dump(processed_events, f, indent=4)
        os.replace(temp_file, PROCESSED_EVENTS_FILE)
    except Exception as e:
        print(f"保存缓存失败: {str(e)}")

_processed_event_ids = load_processed_events()

def verify_cookie(cookie_value: str) -> tuple[bool, str]:
    """
    验证cookies字符串有效性

    Args:
        cookie_value: cookies字符串

    Returns:
        tuple[bool, str]: (是否有效, 错误信息)
    """
    if not isinstance(cookie_value, str) or "__Secure-next-auth.session-token" not in cookie_value:
        return False, "Cookies 值无效，必须包含 __Secure-next-auth.session-token 字段。"
    return True, None

def verify_auth_token(auth_token_value: str) -> tuple[bool, str]:
    """
    验证auth_token字符串有效性

    Args:
        auth_token_value: auth_token字符串

    Returns:
        tuple[bool, str]: (是否有效, 错误信息)
    """
    if not isinstance(auth_token_value, str) or not auth_token_value.strip():
        return False, "Auth Token 值无效，不能为空。"
    if "Bearer " not in auth_token_value:
        return False, "Auth Token 值无效，必须以 'Bearer ' 开头 (注意 Bearer 后有一个空格)。"
    return True, None

handle_error = {
    "cookies": verify_cookie,
    "auth_token": verify_auth_token,
}


def update_config_value(config_file_path, variable_name, new_value):
    """
    更新配置文件中指定变量的值，并自动更新 expires_at。

    Args:
        config_file_path: 配置文件路径.
        variable_name: 要更新的变量名 (cookies 或 auth_token).
        new_value: 变量的新值.

    Returns:
        tuple: (更新是否成功: bool, 回复消息: str).
               成功时返回 (True, 成功消息)，失败时返回 (False, 错误消息).
    """
    if variable_name in handle_error:
        is_valid, err_msg = handle_error[variable_name](new_value)
        if not is_valid:
            return False, f"'{variable_name}' 更新失败: {err_msg}"

    try:
        with open(config_file_path, 'r', encoding='utf-8') as f:
            config_data = json.load(f)

        if variable_name in config_data and config_data[variable_name] == new_value:
            return False, f"变量 '{variable_name}' 的新值与旧值相同，无需更新。"

        config_data[variable_name] = new_value
        expires_at_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=8))) + datetime.timedelta(hours=8)
        config_data["expires_at"] = expires_at_time.isoformat()

        with open(config_file_path, 'w', encoding='utf-8') as f:
            json.dump(config_data, f, indent=2, ensure_ascii=False)

        return True, f"'{variable_name}' 已成功更新，令牌有效至 {expires_at_time.strftime('%Y-%m-%d %H:%M')}"

    except FileNotFoundError:
        return False, f"配置文件 '{config_file_path}' 未找到，更新失败。"
    except json.JSONDecodeError:
        return False, f"配置文件 '{config_file_path}' JSON 格式错误，更新失败。"
    except Exception as e:
        return False, f"更新配置文件时发生未知错误: {e}"


def convert_to_opus(input_path: str, ffmpeg_path: str = None, output_dir: str = None, overwrite: bool = False) -> tuple:
    """
    将音频文件转换为opus格式。

    Args:
        input_path: 输入音频文件路径
        ffmpeg_path: ffmpeg可执行文件路径，默认使用系统PATH中的ffmpeg
        output_dir: 输出目录，默认与输入文件相同目录
        overwrite: 是否覆盖已存在的输出文件

    Returns:
        tuple: (输出文件路径, 音频时长(毫秒))

    Raises:
        FileNotFoundError: ffmpeg或输入文件不存在
        RuntimeError: 转换失败
        FileExistsError: 输出文件已存在且不允许覆盖
    """
    input_path = Path(input_path)
    ffmpeg_path = Path(ffmpeg_path) if ffmpeg_path else None

    if ffmpeg_path and not ffmpeg_path.exists():
        raise FileNotFoundError(f"ffmpeg未找到: {ffmpeg_path}")
    elif not ffmpeg_path and not shutil.which("ffmpeg"):
        raise RuntimeError("未找到系统ffmpeg，请安装或指定自定义路径")

    if not input_path.exists():
        raise FileNotFoundError(f"输入文件不存在: {input_path}")

    output_path = Path(output_dir or input_path.parent) / f"{input_path.stem}.opus"
    if output_path.exists() and not overwrite:
        raise FileExistsError(f"输出文件已存在: {output_path}")

    cmd = [
        str(ffmpeg_path) if ffmpeg_path else "ffmpeg",
        "-i", str(input_path),
        "-strict", "-2",
        "-acodec", "opus",
        "-ac", "1",
        "-ar", "48000",
        "-y" if overwrite else "-n",
        str(output_path)
    ]

    try:
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
        duration = None
        for line in process.stdout:
            if "Duration:" in line:
                time_str = line.split("Duration: ")[1].split(",")[0].strip()
                h, m, s = time_str.split(":")
                duration = int((int(h)*3600 + int(m)*60 + float(s)) * 1000)

        process.wait()
        if process.returncode != 0:
            raise RuntimeError("转换失败")

        return str(output_path), duration

    except subprocess.CalledProcessError as e:
        raise RuntimeError(f"转换失败: {e.stderr.strip()}") from e

def send_message_to_feishu(message_text: str, receive_id: str = "ou_bb1ec32fbd4660b4d7ca36b3640f6fde") -> None:
    """
    发送文本消息到飞书。

    Args:
        message_text: 要发送的消息文本
        receive_id: 接收者的open_id，默认为预设ID
    """
    content = json.dumps({"text": message_text})
    request = (
        CreateMessageRequest.builder()
        .receive_id_type("open_id")
        .request_body(
            CreateMessageRequestBody.builder()
            .receive_id(receive_id)
            .msg_type("text")
            .content(content)
            .build()
        )
        .build()
    )
    response = client.im.v1.message.create(request)
    if not response.success():
        print(f"发送消息失败: {response.code}, {response.msg}")
    else:
        print("消息已发送")

def send_daily_schedule():
    #  **TODO:**  获取当日日程信息 (例如从日历 API 或本地文件读取)
    schedule_text = "今日日程:\n- 上午 9:00  会议\n- 下午 2:00  代码 Review" #  替换为实际日程
    send_message_to_feishu(schedule_text)

def send_bilibili_updates():
    # **TODO:** 获取 B 站更新信息 (例如调用 B站 API)
    update_text = "B站更新:\n- XXX up主发布了新视频：[视频标题](视频链接)" #  替换为实际更新信息
    send_message_to_feishu(update_text)

def send_daily_summary():
    # **TODO:**  生成每日总结内容 (例如分析日志、统计数据)
    summary_text = "每日总结:\n- 今日完成 XX 任务\n- 发现 XX 问题" #  替换为实际总结内容
    send_message_to_feishu(summary_text)

def do_p2_im_message_receive_v1(data: P2ImMessageReceiveV1) -> None:
    """
    处理飞书机器人接收到的消息事件。

    Args:
        data: 飞书消息事件数据对象，包含消息内容、发送者等信息
    """
    # 获取事件基本信息
    # print(f"接收到消息事件，data 对象信息: {data.__dict__}") #  打印 data 对象的所有属性
    # print(f"接收到消息事件，data.event 对象信息: {data.event.__dict__}") #  打印 data.event 对象的所有属性
    # print(f"接收到消息事件，data.header 对象信息: {data.header.__dict__}") #  打印 data.header 对象的所有属性
    event_time = data.header.create_time or time.time()
    event_id = data.header.event_id

    # 检查事件是否已处理过
    if event_id in _processed_event_ids.keys():
        print(f"超时响应的事件，目前认定不需要处理: {event_id}")
        return

    # 记录新事件
    _processed_event_ids[event_id] = event_time
    save_processed_events()

    def send_message(msg_type: str, content: str) -> None:
        """发送消息的通用函数"""
        if data.event.message.chat_type == "p2p":
            request = CreateMessageRequest.builder().receive_id_type("chat_id").request_body(
                CreateMessageRequestBody.builder()
                .receive_id(data.event.message.chat_id)
                .msg_type(msg_type)
                .content(content)
                .build()
            ).build()
            response = client.im.v1.chat.create(request)
        else:
            request = ReplyMessageRequest.builder().message_id(data.event.message.message_id).request_body(
                ReplyMessageRequestBody.builder()
                .content(content)
                .msg_type(msg_type)
                .build()
            ).build()
            response = client.im.v1.message.reply(request)

        if not response.success():
            print(f"消息发送失败: {response.code} - {response.msg}")
        return response

    def handle_image_upload(image_path: str) -> tuple[str, str]:
        """处理图片上传,返回消息类型和内容"""
        with open(image_path, "rb") as image_file:
            upload_response = client.im.v1.image.create(
                CreateImageRequest.builder()
                .request_body(CreateImageRequestBody.builder().image_type("message").image(image_file).build())
                .build()
            )
            if upload_response.success() and upload_response.data and upload_response.data.image_key:
                return "image", json.dumps({"image_key": upload_response.data.image_key})
            return "text", json.dumps({"text": f"图片上传失败: {upload_response.code} - {upload_response.msg}"})

    def handle_audio_upload(audio_path: str) -> tuple[str, str]:
        """处理音频上传,返回消息类型和内容"""
        opus_path, duration_ms = convert_to_opus(audio_path, custom_ffmpeg, overwrite=True)
        with open(opus_path, "rb") as audio_file:
            opus_filename = Path(audio_path).stem + '.opus'
            upload_response = client.im.v1.file.create(
                CreateFileRequest.builder()
                .request_body(
                    CreateFileRequestBody.builder()
                    .file_type("opus")
                    .file_name(opus_filename)
                    .duration(str(int(duration_ms)))
                    .file(audio_file)
                    .build()
                ).build()
            )
            if upload_response.success() and upload_response.data and upload_response.data.file_key:
                return "audio", json.dumps({"file_key": upload_response.data.file_key})
            return "text", json.dumps({"text": f"音频上传失败: {upload_response.code} - {upload_response.msg}"})

    def handle_ai_image_generation(prompt: str = None, image_input: dict = None) -> tuple[str, str]:
        """
        处理AI生图或图片处理请求

        Args:
            prompt: 文本提示词,用于AI生图
            image_input: 图片输入参数,用于图片处理

        Returns:
            tuple: 消息类型和内容
        """
        predict_kwargs = {
            "image_input1": None,
            "image_input2": None,
            "style_key": "贺卡",
            "additional_text": "",
            "api_name": "/generate_images"
        }
        if image_input:
            predict_kwargs["image_input1"] = image_input

        elif prompt:
            predict_kwargs["additional_text"] = "/img " + prompt


        result = gradio_client.predict(**predict_kwargs)

        if not isinstance(result, tuple) or len(result) == 0:
            return "text", json.dumps({"text": "图片失败，已经通知管理员修复咯！"})

        if all(x is None for x in result):
            return "text", json.dumps({"text": "图片生成失败了，建议您换个提示词再试试"})

        for image_path in result:
            if image_path is not None:
                msg_type, content = handle_image_upload(image_path)
                if msg_type == "image":
                    send_message(msg_type, content)

        return None, None

    def handle_message_resource(message_id: str, file_key: str, resource_type: str) -> tuple[bytes, str, dict]:
        """
        获取消息中的资源文件

        Args:
            message_id: 消息ID
            file_key: 文件key
            resource_type: 资源类型

        Returns:
            tuple: 文件内容、文件名和元信息
        """
        request = GetMessageResourceRequest.builder() \
            .message_id(message_id) \
            .file_key(file_key) \
            .type(resource_type) \
            .build()

        response = client.im.v1.message_resource.get(request)

        if not response.success():
            print(f"获取资源文件失败: {response.code} - {response.msg}")
            return None, None, None

        file_content = response.file.read()
        file_name = response.file_name
        meta = {
            "size": len(file_content),
            "mime_type": response.file.content_type if hasattr(response.file, "content_type") else None
        }

        return file_content, file_name, meta

    bili_videos = [
        {"title": "【中字】蔚蓝的难度设计为什么这么完美", "bvid": "BV1BAABeKEoJ"},
        {"title": "半年减重100斤靠什么？首先排除毅力 | 果壳专访", "bvid": "BV1WHAaefEEV"},
        {"title": "作为普通人我们真的需要使用Dify吗？", "bvid": "BV16fKWeGEv1"},
    ]

    # 获取消息内容
    msg_type = "text"
    content = None
    user_msg = None

    # 解析消息内容
    if data.event.message.message_type == "text":
        user_msg = json.loads(data.event.message.content)["text"]
    elif data.event.message.message_type == "image":
        image_content = json.loads(data.event.message.content)
        if "image_key" in image_content:
            file_content, file_name, meta = handle_message_resource(
                data.event.message.message_id,
                image_content["image_key"],
                "image"
            )
            if file_content:
                base64_image = base64.b64encode(file_content).decode('utf-8')
                image_url = f"data:{meta['mime_type']};base64,{base64_image}"
                image_input = {
                    "path": None,
                    "url": image_url,
                    "size": meta["size"],
                    "orig_name": file_name,
                    "mime_type": meta["mime_type"],
                    "is_stream": False,
                    "meta": {}
                }
                send_message("text", json.dumps({"text": "正在转换图片风格，请稍候..."}))

                try:
                    msg_type, content = handle_ai_image_generation(image_input=image_input)
                except Exception as e:
                    content = json.dumps({"text": f"AI 图片处理错误: {str(e)}"})
            else:
                content = json.dumps({"text": "获取图片资源失败"})
        else:
            content = json.dumps({"text": "图片消息格式错误"})
    elif data.event.message.message_type == "audio":
        audio_content = json.loads(data.event.message.content)
        if "file_key" in audio_content:
            file_content, file_name, meta = handle_message_resource(
                data.event.message.message_id,
                audio_content["file_key"],
                "file"
            )
            if file_content:
                content = json.dumps({"text": "这是一个待开发的音频处理流程"})
            else:
                content = json.dumps({"text": "获取音频资源失败"})
        else:
            content = json.dumps({"text": "音频消息格式错误"})
    else:
        content = json.dumps({"text": "解析消息失败，请发送文本消息"})

    # 处理文本消息
    if not content and user_msg:
        if "你好" in user_msg:
            content = json.dumps({"text": "你好呀！有什么我可以帮你的吗？"})

        elif user_msg.startswith(UPDATE_CONFIG_TRIGGER):
            if data.event.sender.sender_id.open_id != os.getenv("ADMIN_ID"):
                content = json.dumps({"text": f"Received message:{user_msg}"})
            else:
                command_parts = user_msg[len(UPDATE_CONFIG_TRIGGER):].strip().split(maxsplit=1)
                if len(command_parts) == 2:
                    variable_name = command_parts[0].strip()
                    new_value = command_parts[1].strip()
                    if variable_name in SUPPORTED_VARIABLES:
                        success, reply_text_update = update_config_value(CONFIG_FILE_PATH, variable_name, new_value)
                        content = json.dumps({"text": reply_text_update})
                    else:
                        content = json.dumps({"text": f"不支持更新变量 '{variable_name}'，只能更新: {', '.join(SUPPORTED_VARIABLES)}"})
                else:
                    content = json.dumps({"text": f"格式错误，请使用 '{UPDATE_CONFIG_TRIGGER} 变量名 新值' 格式，例如：{UPDATE_CONFIG_TRIGGER} cookies xxxx"})

        elif "帮助" in user_msg:
            content = json.dumps({
                "text": "<b>我可以帮你做这些事情：</b>\n\n"
                        "1. <b>图片风格转换</b>\n"
                        "上传任意照片，我会把照片转换成<i>剪纸贺卡</i>风格的图片\n\n"
                        "2. <b>视频推荐</b>\n"
                        "输入\"B站\"或\"视频\"，我会<i>随机推荐</i>B站视频给你\n\n"
                        "3. <b>图片分享</b>\n"
                        "输入\"图片\"或\"壁纸\"，我会分享<u>精美图片</u>\n\n"
                        "4. <b>音频播放</b>\n"
                        "输入\"音频\"，我会发送<u>语音消息</u>\n\n"
                        "<at user_id=\"all\"></at> 随时输入\"帮助\"可以<i>再次查看</i>此菜单"
            })

        elif "富文本" in user_msg:
            try:
                msg_type, content = handle_image_upload(r"E:\Download\image (4).webp")
                if msg_type == "image" and content:
                    image_key = json.loads(content)["image_key"]
                    msg_type = "post"
                    content = json.dumps({
                        "zh_cn": {
                            "title": "富文本示例",
                            "content": [
                                [
                                    {"tag": "text", "text": "第一行:", "style": ["bold", "underline"]},
                                    {"tag": "a", "href": "https://open.feishu.cn", "text": "飞书开放平台", "style": ["italic"]},
                                    {"tag": "at", "user_id": "all", "style": ["lineThrough"]}
                                ],
                                [{"tag": "img", "image_key": image_key}],
                                [
                                    {"tag": "text", "text": "代码示例:"},
                                    {"tag": "code_block", "language": "PYTHON", "text": "print('Hello World')"}
                                ],
                                [{"tag": "hr"}],
                                [{"tag": "md", "text": "**Markdown内容**\n- 列表项1\n- 列表项2\n```python\nprint('代码块')\n```"}]
                            ]
                        }
                    })
            except Exception as e:
                content = json.dumps({"text": f"富文本处理错误: {str(e)}"})

        elif "B站" in user_msg or "视频" in user_msg:
            video = random.choice(bili_videos)
            content = json.dumps({"text": f"为你推荐B站视频：\n{video['title']}\nhttps://www.bilibili.com/video/{video['bvid']}"})

        elif "图片" in user_msg or "壁纸" in user_msg:
            send_message("text", json.dumps({"text": "正在处理图片，请稍候..."}))
            try:
                msg_type, content = handle_image_upload(r"E:\Download\image (4).webp")
            except Exception as e:
                content = json.dumps({"text": f"图片处理错误: {str(e)}"})

        elif "生图" in user_msg or "AI画图" in user_msg:
            send_message("text", json.dumps({"text": "正在生成图片，请稍候..."}))
            try:
                prompt = user_msg.replace("生图", "").replace("AI画图", "").strip()
                msg_type, content = handle_ai_image_generation(prompt)
            except Exception as e:
                content = json.dumps({"text": f"AI生图错误: {str(e)}"})

        elif "音频" in user_msg:
            send_message("text", json.dumps({"text": "正在处理音频，请稍候..."}))
            try:
                msg_type, content = handle_audio_upload(r"C:\Users\A\Project_NovelGame\notebooks\output.mp3")
            except Exception as e:
                content = json.dumps({"text": f"音频处理错误: {str(e)}"})

        elif "配音" in user_msg:
            send_message("text", json.dumps({"text": "正在生成配音，请稍候..."}))
            try:
                tts_text = user_msg.split("配音", 1)[1].strip()

                # 直接获取音频字节流
                audio_data = coze_tts.generate(tts_text)
                if not audio_data:
                    raise ValueError("TTS故障，请联系管理员检查API配置")

                # 使用内存文件避免磁盘IO
                with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as tmp:
                    tmp.write(audio_data)
                    tmp.flush()  # 确保数据写入
                    tmp_path = tmp.name

                try:
                    msg_type, content = handle_audio_upload(tmp_path)
                    send_message(msg_type, content)
                except Exception as upload_err:
                    raise Exception(f"音频上传失败: {str(upload_err)}")
                finally:
                    # 清理临时文件
                    if os.path.exists(tmp_path):
                        os.unlink(tmp_path)

            except Exception as e:
                error_msg = str(e)
                if "音频上传失败" not in error_msg:
                    error_msg = f"配音生成失败: {error_msg}"
                send_message("text", json.dumps({"text": error_msg}))

        else:
            content = json.dumps({"text": f"收到你发送的消息：{user_msg}\nReceived message:{user_msg}"})

    # 发送最终消息
    if content:
        send_message(msg_type, content)

# 注册事件处理
event_handler = lark.EventDispatcherHandler.builder("", "").register_p2_im_message_receive_v1(do_p2_im_message_receive_v1).build()

# 创建客户端
client = lark.Client.builder().app_id(lark.APP_ID).app_secret(lark.APP_SECRET).build()
wsClient = lark.ws.Client(lark.APP_ID, lark.APP_SECRET, event_handler=event_handler, log_level=lark.LogLevel.INFO)


async def main_jupyter():
    """Jupyter环境下的主函数，建立WebSocket连接并保持运行"""
    await wsClient._connect()
    print("WebSocket 连接已建立，等待接收消息...")

    # now = datetime.datetime.now()
    # test_time_schedule = (now + datetime.timedelta(seconds=5)).strftime("%H:%M:%S") # 15秒后的时间
    # test_time_bilibili = (now + datetime.timedelta(seconds=15)).strftime("%H:%M:%S") # 30秒后的时间
    # test_time_summary = (now + datetime.timedelta(seconds=25)).strftime("%H:%M:%S")  # 45秒后的时间

    # schedule.every().day.at(test_time_schedule).do(send_daily_schedule) #  每天在 test_time_schedule 发送日程
    # schedule.every().day.at(test_time_bilibili).do(send_bilibili_updates) #  每天在 test_time_bilibili 发送 B站更新
    # schedule.every().day.at(test_time_summary).do(send_daily_summary)  #  每天在 test_time_summary 发送每日总结

    # schedule.every().day.at("07:30").do(send_daily_schedule) #  每天 7:30 发送日程
    # schedule.every().day.at("15:00").do(send_bilibili_updates) #  每天 15:00 发送 B站更新
    # schedule.every().day.at("22:00").do(send_daily_summary)  #  每天 22:00 发送每日总结
    try:
        while True:
            schedule.run_pending()
            await asyncio.sleep(10)
    except KeyboardInterrupt:
        print("程序已停止")
    finally:
        await wsClient._disconnect()

if __name__ == "__main__":
    pass

await main_jupyter()